In [19]:
import os
import numpy as np
import cv2
from joblib import load, Parallel, delayed
from skimage.color import deltaE_ciede2000 as deltaE
from matplotlib import pyplot as plt


In [8]:
c_map = load("colormap.joblib")

In [9]:
means = [x['mean'] for x in c_map]

In [17]:
def get_match(means, target_patch):
    target_mean = np.mean(target_patch, axis=0)
    delta_map = np.array(list(map(lambda candidate:deltaE(target_mean,candidate, kL=1), means)))
    candidate = np.random.choice(np.where(delta_map == np.amin(delta_map))[0])
    return candidate

In [18]:
get_match(means, [[10, 62, 70],[10, 62, 70]])

733

TypeError: images is not a numpy array, neither a scalar

<Figure size 432x288 with 0 Axes>